In [1]:
# import necessary libraries
%reset
import os
import json
import datetime
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
# import thin_plate_spline_warp

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [2]:
name = "test_model"

In [3]:
def create_workspace_folders():
    date = datetime.datetime.now()
    workspace = "workspace\{}-{}".format(date.strftime("%Y_%m_%d"), name)

    create_folders = ['workspace', workspace]
    # create workspace
    for folder in create_folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
    
    return workspace

workspace_folderpath = create_workspace_folders()

In [4]:
# Load necessary file structures via config file
with open('config.json') as f:
    config = json.load(f)

In [5]:
# STEP 1: Import all the necessary data
def parse_mipav_data(config, cell_key, strain_folderpath):
    # define output variables
    seam_cell_output = {}
    annotation_output = {} # structured cell_name: pandas dataframe (timepoint, x, y, z)
    errors = [] # note potential errors in volumes
    
    # determine default or options specific to strain
    folderpaths = config['settings']['folderpaths']
    for folder_data in folderpaths.keys():
        try:
            folderpaths[folder_data] = cell_key['folderpaths'][folder_data]
            print('Using specific {} folder structure'.format(folder_data))
        except:
            # print('Using default {} folder structure'.format(folder_data))
            pass
    
    # find the numbers of the folders
    start_vol = int(cell_key['start'])
    end_vol = int(cell_key['end'])
    all_folderpaths = folderpaths.copy()
    for vol_idx, vol_num in enumerate(range(start_vol, end_vol+1)):
        # define the specific volume number folder
        vol_folder = all_folderpaths['data_folderpath'].replace('#', str(vol_num))
        vol_folderpath = os.path.join(
            strain_folderpath, all_folderpaths['side'], vol_folder)
        
        # check if the folder exists
        if not os.path.isdir(vol_folderpath):
            # print(vol_folderpath, ' not found.')
            continue
        else:
            # print("Found {} timepoint {}.".format(cell_key['name'], vol_num))
            pass
        
        # define individual, full filepaths from drive
        all_filepaths_folderpaths = {}
        for full_folderpaths_key in all_folderpaths.keys():
            if full_folderpaths_key != "side" and \
                full_folderpaths_key != "data_folderpath":
                all_filepaths_folderpaths[full_folderpaths_key] = os.path.join(
                    vol_folderpath, all_folderpaths[full_folderpaths_key])
                # print(all_filepaths_folderpaths[full_folderpaths_key])

                # check to see if exists
                if not os.path.isfile(all_filepaths_folderpaths[full_folderpaths_key]):
                    error_msg = "FILE DOES NOT EXIST: {}".format(
                        all_filepaths_folderpaths[full_folderpaths_key])
                    errors.append(error_msg)
                    print(error_msg)
        
        ## get individual data by cell in pandas format
        try:
            # import straighted seam cells
            st_seam_cells_fp = all_filepaths_folderpaths['straightened_seam_cells']
            st_seam_cells = pd.read_csv(st_seam_cells_fp)
            # import straightened annotations
            st_annotations_fp = all_filepaths_folderpaths['straightened_annotations']
            st_annotations = pd.read_csv(st_annotations_fp)
        except:
            continue # file errors should already be logged.
            
        # SEAM CELLS: combine into a single data structure and catch data errors
        # go through each row of the file and log the appropriate information
        for idx, seam_row in st_seam_cells.iterrows():
            seam_cell_name = seam_row['name'].upper() # seam cells are upper case? just stay consistent
            if seam_cell_name not in seam_cell_output.keys():
                seam_cell_output[seam_cell_name] = {}
                seam_cell_output[seam_cell_name]['timepoints'] = []
                seam_cell_output[seam_cell_name]['coordinates'] = []
                                
            # append to appropriate seam cell
            seam_cell_coords = seam_row[['x_voxels','y_voxels','z_voxels']].values.flatten().tolist()
            seam_cell_output[seam_cell_name]['coordinates'].append(seam_cell_coords)
            seam_cell_output[seam_cell_name]['timepoints'].append(vol_num)
        
        # ANNOTATIONS: combine into a single data structure and catch data errors
        try:
            for cell_id in cell_key['mapping'].keys():
                cell_name = cell_key['mapping'][cell_id].lower()
                if cell_name not in annotation_output.keys():
                    annotation_output[cell_name] = {}
                    annotation_output[cell_name]['timepoints'] = []
                    annotation_output[cell_name]['coordinates'] = []

                # check if the cell name exists in volume
                # print(vol_folderpath, st_annotations['name'], cell_id)
                cell_row = st_annotations.loc[st_annotations['name'] == cell_id]
                if not cell_row.empty:
                    # try to catch a few errors
                    if cell_row.shape[0] > 1:
                        error_msg = "DATA ERROR: Ignoring identical cell IDs ({}) for cell {} in {}".format(
                            cell_id, cell_name, st_annotations_fp)
                        errors.append(error_msg)
                        print(error_msg)
                        continue
                    
                    # if there is nothing wrong, then proceed
                    cell_coords = cell_row[['x_voxels','y_voxels','z_voxels']].values.flatten().tolist()
                    annotation_output[cell_name]['coordinates'].append(cell_coords)
                    annotation_output[cell_name]['timepoints'].append(vol_num)
        except:
            error_msg = "DATA ERROR: Failure to read file {}".format(st_annotations_fp)
            errors.append(error_msg)
            print(error_msg)
                
    return seam_cell_output, annotation_output, errors

def convert_cell_key_csv2json(csv_filepath):
    cell_key_json = {}
    # load in csv file
    cell_key = pd.read_csv(csv_filepath, header=None, engine='python')
    
    ## get necessary information
    cell_key_json['name'] = str(cell_key.iloc[0,0])
    cell_key_json['start'] = int(cell_key.iloc[1,0])
    cell_key_json['end'] = int(cell_key.iloc[1,1])
    try:
        cell_key_json['outliers'] = cell_key.iloc[2].astype(int).values.tolist()
    except:
        cell_key_json['outliers'] = [] # if there are none
    
    # grab mapping
    cell_key_json['mapping'] = {}
    for row_idx in range(3, cell_key.shape[0]): # mapping starts row idx 3
        cell_id = str(cell_key.iloc[row_idx,0])
        cell_name = str(cell_key.iloc[row_idx,1])
        cell_key_json['mapping'][cell_id] = cell_name

    return cell_key_json
    
def get_cell_key(pos_folderpath):
    # return the cell key in dict/json format and convert any existing 
    # cell key csv into json if the json version doesn't exist.
    cell_key_filepath_csv = os.path.join(pos_folderpath, 'CellKey.csv')
    cell_key_filepath_json = os.path.join(pos_folderpath, 'cell_key.json')
    
    # if the json exists, use it
    if os.path.isfile(cell_key_filepath_json):
        with open(cell_key_filepath_json) as f:
            return json.load(f)
    elif os.path.isfile(cell_key_filepath_csv):
        cell_key_json = convert_cell_key_csv2json(cell_key_filepath_csv)
        cell_key_json_dump = json.dumps(cell_key_json, sort_keys=True, indent=4)
        with open(cell_key_filepath_json, "w") as f: 
            f.write(cell_key_json_dump)
        return cell_key_json
    else:
        cell_key_error_msg = 'Missing cell key: {}'.format(cell_key_filepath_json)
        print(cell_key_error_msg)
        return None
                
strain_info = config['data']['strains']
compiled_data = {}
for strain in strain_info:
    if strain['include']:
        if strain['name'] not in compiled_data.keys():
            compiled_data[strain['name']] = {}
                
        for pos_folderpath in strain['folderpaths']:
            print('CURRENTLY LOADING: {}'.format(pos_folderpath))
            # find and load cell key
            cell_key = get_cell_key(pos_folderpath)
            if not cell_key:
                continue
                
            # get data
            seam_cells, annotations, strain_errors = parse_mipav_data(
                config, cell_key, pos_folderpath)
            
            # combine data
            pos_name = cell_key['name']
            compiled_data[strain['name']][pos_name] = {}
            compiled_data[strain['name']][pos_name]['cell_key'] = cell_key
            compiled_data[strain['name']][pos_name]['seam_cells'] = seam_cells
            compiled_data[strain['name']][pos_name]['annotations'] = annotations
            compiled_data[strain['name']][pos_name]['errors'] = strain_errors

# save information as intermediate step in workspace
compiled_json = json.dumps(compiled_data, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '1_compiled_data.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json) 

CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\120619_Pos2\Decon_reg
DATA ERROR: Ignoring identical cell IDs (C3) for cell rigl in Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\120619_Pos2\Decon_reg\RegB\Decon_reg_81\Decon_reg_81_results\straightened_annotations\straightened_annotations.csv
DATA ERROR: Ignoring identical cell IDs (D4) for cell hyp6 in Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\120619_Pos2\Decon_reg\RegB\Decon_reg_88\Decon_reg_88_results\straightened_annotations\straightened_annotations.csv
DATA ERROR: Ignoring identical cell IDs (D4) for cell hyp6 in Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\120619_Pos2\Decon_reg\RegB\Decon_reg_100\Decon_reg_100_results\straightened_annotations\straightened_annotations.csv
DATA ERROR: Ignoring identical cell IDs (D4) for cell hyp6 in Y:\RyanC\Cell Tracking Project\OD1599_NU\OD1599_MostRecent\120619_Pos2\Decon_reg\RegB\Decon_reg_101\Decon_reg_101_results\straig

FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_51\Decon_reg_51_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_52\Decon_reg_52_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_53\Decon_reg_53_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_54\Decon_reg_54_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_55\Decon_reg_55_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_56\Decon_reg_56_

FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_98\Decon_reg_98_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_98\Decon_reg_98_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_98\Decon_reg_98_results\straightened_lattice\straightened_lattice.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_98\Decon_reg_98_results\seam_cell_final\seam_cells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_98\Decon_reg_98_results\lattice_final\lattice.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos0\RegB\Decon_reg_99\Decon_reg_99_results\straightened_annotations\straightened_annotations.csv
F

C:\Users\NIBIB Analysis\Anaconda3\lib\site-packages\pandas\core\ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


DATA ERROR: Failure to read file Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_19\Decon_reg_19_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_28\Decon_reg_28_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_28\Decon_reg_28_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_29\Decon_reg_29_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_29\Decon_reg_29_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_30\Decon_reg

FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_69\Decon_reg_69_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_70\Decon_reg_70_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_70\Decon_reg_70_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_71\Decon_reg_71_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_72\Decon_reg_72_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_72\Decon_reg_72_results\stra

FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_110\Decon_reg_110_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_110\Decon_reg_110_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_111\Decon_reg_111_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_112\Decon_reg_112_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_113\Decon_reg_113_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\KP9305_NU\073018_KP9305_NU\Pos4\RegB\Decon_reg_114\Decon_reg_114_resu

CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\RW10752_NU\031219_Pos2
DATA ERROR: Ignoring identical cell IDs (C2Q) for cell posterior_arcade_dr in Y:\RyanC\Cell Tracking Project\RW10752_NU\031219_Pos2\RegB\Decon_reg_89\Decon_reg_89_results\straightened_annotations\straightened_annotations.csv
DATA ERROR: Ignoring identical cell IDs (C2Q) for cell posterior_arcade_dr in Y:\RyanC\Cell Tracking Project\RW10752_NU\031219_Pos2\RegB\Decon_reg_90\Decon_reg_90_results\straightened_annotations\straightened_annotations.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\RW10752_NU\031219_Pos2\RegB\Decon_reg_93\Decon_reg_93_results\integrated_annotation\annotations.csv
CURRENTLY LOADING: Y:\RyanC\Cell Tracking Project\RW10752_NU\031219_Pos1
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\RW10752_NU\031219_Pos1\RegB\Decon_reg_15\Decon_reg_15_results\straightened_seamcells\straightened_seamcells.csv
FILE DOES NOT EXIST: Y:\RyanC\Cell Tracking Project\RW10752_NU\031219_Pos1\RegB\Decon_re

In [8]:
# STEP 2: check for outliers, uses parameters defined in config.json
def hampel_filter(input_series, window=5, n_sigmas=1):
    # ensure the data is flattened
    input_series = np.array(input_series).flatten()
    
    # returns filtered timepoints and coordinates along
    n = len(input_series)
    filtered_series = input_series.copy()
    k = 1.4826 # scale factor for Gaussian distribution
    
    outliers_idxs = []
    outliers_filtered = []
    for i in range(window_size, n - window_size):
        x0 = np.median(
            input_series[(i - window_size):(i + window_size)])
        S0 = k * np.median(np.abs(
            input_series[(i - window_size):(i + window_size)] - x0))
        if (np.abs(input_series[i] - x0) > n_sigmas * S0):
            filtered_series[i] = x0 # replaces outlier with median
            outliers_idxs.append(i) # logs outlier index
            outliers_filtered.append(x0) # the value that replaces outlier
    
    return filtered_series, outliers_idxs, outliers_filtered


window_size = config['settings']['outlier_removal']['window_size']
n_stdev = config['settings']['outlier_removal']['n_stdev']
compiled_data_no_outliers = compiled_data.copy()
for strain in compiled_data.keys():
    for pos in compiled_data[strain].keys():

        # determine outliers for both seam cells and annotations
        for cells_type in ['seam_cells', 'annotations']:
            for cell in compiled_data[strain][pos][cells_type].keys():
                coordinates = np.array(compiled_data[strain][pos][cells_type][cell]['coordinates'])

                # for some reason if it's empty, then skip it
                if coordinates.size == 0:
                    continue

                # separate by x,y,z (index 0-2) and filter
                outlier_idxs = {} # set automatically removes repeats
                for dim_idx in range(3):
                    coord_dim_data = coordinates[:, dim_idx].copy()
                    data_filtered, outlier_idx, outlier_val = hampel_filter(coord_dim_data)
                    coordinates[:, dim_idx] = data_filtered

                # replace time series outliers using median
                compiled_data_no_outliers[strain][pos][cells_type][cell]['coordinates'] = coordinates.tolist()

# save information as intermediate step in workspace
compiled_json = json.dumps(compiled_data_no_outliers, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '2_compiled_data_no_outliers.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)       

In [13]:
# STEP 3: interpolate each to appropriate time scale
compiled_data_interp = compiled_data_no_outliers.copy()
seam_cells_on = config['settings']['interpolation']['seam_cells_on'] # in minutes
total_len = config['settings']['interpolation']['total_min'] # in minutes
interp_method = config['settings']['interpolation']['method']
new_timepoints = np.linspace(0, total_len)

for strain in compiled_data_no_outliers.keys():
    for pos in compiled_data_no_outliers[strain].keys():
        
        # interpolate for both seam cells and annotations
        for cells_type in ['seam_cells', 'annotations']:
            for cell in compiled_data_no_outliers[strain][pos][cells_type].keys():
                print(strain, pos, cells_type, cell)
                
                timepoints = np.array(compiled_data_no_outliers[strain][pos][cells_type][cell]['timepoints'])
                coordinates = np.array(compiled_data_no_outliers[strain][pos][cells_type][cell]['coordinates'])

                # for some reason if it's empty, then skip it
                if coordinates.size == 0:
                    continue
                
                # handle seam cells and annotations differently because 
                # some seam cells appear after twitching begins
                if cells_type == 'seam_cells':
                    starting_idx = round(seam_cells_on[cell] * total_len)
                elif cells_type == 'annotations':
                    starting_idx = 0
                
                new_coordinates = np.zeros((total_len, 3))
                new_timepoints = np.arange(total_len)
                
                # interpolate one dimension at a time
                for dim_idx in range(3):
                    coord_dim_data = coordinates[:, dim_idx].copy()
                    # rescale time points to range
                    cell_sp_timepoints = np.arange(total_len - starting_idx) # scaled time
                    cell_sp_rescaled = timepoints - min(timepoints)
                    cell_sp_rescaled = cell_sp_rescaled/max(cell_sp_rescaled) * (total_len - starting_idx)
                    
                    interp = interp1d(cell_sp_rescaled, coord_dim_data,
                                     kind=interp_method) # get interp as if from 0, but shift below
                    new_coordinates[starting_idx:, dim_idx] = interp(cell_sp_timepoints) # shifted

                # replace time series outliers using median
                compiled_data_interp[strain][pos][cells_type][cell]['coordinates'] = new_coordinates.tolist()
                compiled_data_interp[strain][pos][cells_type][cell]['timepoints'] = new_timepoints.tolist()
                
# save information as intermediate step in workspace
compiled_json = json.dumps(compiled_data_interp, sort_keys=True, indent=4)
compiled_json_filepath = os.path.join(
    workspace_folderpath, '3_compiled_data_interpolation.json')
with open(compiled_json_filepath, "w") as f: 
    f.write(compiled_json)     

OD1599_NU OD1599_NU_1206_Pos2 seam_cells H0L
OD1599_NU OD1599_NU_1206_Pos2 seam_cells H0R
OD1599_NU OD1599_NU_1206_Pos2 seam_cells H1L
OD1599_NU OD1599_NU_1206_Pos2 seam_cells H1R
OD1599_NU OD1599_NU_1206_Pos2 seam_cells H2L
OD1599_NU OD1599_NU_1206_Pos2 seam_cells H2R
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V1L
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V1R
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V2L
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V2R
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V3L
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V3R
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V4L
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V4R
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V5L
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V5R
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V6L
OD1599_NU OD1599_NU_1206_Pos2 seam_cells V6R
OD1599_NU OD1599_NU_1206_Pos2 seam_cells TL
OD1599_NU OD1599_NU_1206_Pos2 seam_cells TR
OD1599_NU OD1599_NU_1206_Pos2 seam_cells QL


ValueError: number of derivatives at boundaries.

In [21]:
# reorganize data by cell because they are all in the same time scale
print(strain, pos)
print(compiled_data_no_outliers[strain][pos][cells_type]['QL'])

OD1599_NU OD1599_NU_1206_Pos2
{'timepoints': [95, 108], 'coordinates': [[51.0, 81.0, 755.0], [50.0, 79.0, 771.0]]}


In [ ]:
# thin plate spline warping

In [ ]:
# perform moving average for each cell

In [ ]:
# spatial average of each strain by cell (because possible variation?)

In [ ]:
# temporal average of all cells (containing multiple samples)

In [ ]:
# output model